In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import plotly.graph_objects as go
import networkx as nx

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import image
import nltk
nltk.download('stopwords')
from keras.preprocessing.text import Tokenizer
from gensim.models import KeyedVectors
from keras.layers import Embedding
from keras.models import Sequential
import tensorflow as tf
from mtcnn import MTCNN


from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model, load_model
import cv2
import os

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Input, Dropout, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data.loaders import SingleLoader, Loader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess
from spektral.utils import normalized_laplacian

import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gsevr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('MELD.Raw/dev_sent_emo.csv',encoding='utf-8')
df['Utterance'] = df.Utterance.str.replace('',"'")
df['gender'] = df.Speaker.apply(lambda x: 'male' if x == 'Ross' or x == 'Joey' or x == 'Chandler' else 'female')
df = df.sample(frac=0.01,ignore_index=True)
df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,gender
0,839,Right. Or you might get everything you've wan...,Phoebe,neutral,neutral,77,19,8,24,"00:18:40,899","00:18:44,778",female
1,812,"I know, And then all of the sudden your Mom po...",Chandler,anger,negative,76,9,3,1,"00:16:49,717","00:16:52,301",male
2,291,"Okay, absolutely!",Chandler,joy,positive,24,4,3,11,"0:20:00,908","0:20:02,180",male
3,824,"I'm sorry, for the last time, why aren't you t...",Phoebe,neutral,neutral,77,4,8,24,"00:17:31,497","00:17:36,084",female
4,99,"Someone I can learn from, someone-someone who'...",Joey,neutral,neutral,9,6,6,3,"00:05:43,676","00:05:47,304",male
5,760,"Tiger! Dragon! Iceberg!! Joseph, did you eve...",Phoebe,anger,negative,72,0,5,11,"00:10:55,738","00:11:03,578",female
6,605,"No, it's been three nights in a row.",Rachel,neutral,neutral,57,8,3,1,"00:01:45,146","00:01:47,064",female
7,603,"Oh my God, has she slept at all?",Phoebe,surprise,negative,57,6,3,1,"00:01:41,559","00:01:44,978",female
8,1039,"No, no, that's the ghost for the attic.",Phoebe,neutral,neutral,97,13,3,20,"00:04:46,619","00:04:50,122",female
9,444,"Well let me think about that, while I remove m...",Chandler,joy,positive,39,2,8,24,"00:01:16,439","00:01:19,484",male


In [15]:
# Load MTCNN model for face detection
detector = MTCNN()

# Load FaceNet model for face embedding extraction
facenet_model = tf.keras.applications.ResNet50(weights='imagenet')

def preprocess_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0 
    return img

def extract_face_embeddings(frame):
    faces = detector.detect_faces(frame)
    face_embeddings = []
    for face in faces:
        x, y, w, h = face['box']
        x1, y1 = max(x, 0), max(y, 0)
        x2, y2 = min(x + w, frame.shape[1]), min(y + h, frame.shape[0])
        cropped_face = frame[y1:y2, x1:x2]
        
        # Preprocess the face for FaceNet model
        preprocessed_face = preprocess_image(cropped_face)
        preprocessed_face = np.expand_dims(preprocessed_face, axis=0)
        
        # Get face embeddings using FaceNet model
        face_embedding = facenet_model.predict(preprocessed_face)
        face_embeddings.append(np.squeeze(face_embedding))
    return face_embeddings

# Read video
folder_path = 'MELD.Raw/dev_splits_complete/'
one_face_videos = {}

for idx, row in df.iterrows():
    # file_name = 'dia23_utt7.mp4'
    file_name = 'dia' + str(row['Dialogue_ID']) + '_utt' + str(row['Utterance_ID']) + '.mp4'
    video_path = folder_path + file_name
    if os.path.isfile(video_path):
        video_capture = cv2.VideoCapture(video_path)

        single_video_embeddings = []  # List to store embeddings for all frames

        frame_counter = 0

        while frame_counter < 3:
            ret, frame = video_capture.read()
            if not ret:
                break

            # check for more than 1 face
            if len(detector.detect_faces(frame)) == 1:

                # Extract face embeddings from each frame
                extracted_embeddings = extract_face_embeddings(frame)

                single_video_embeddings.append(extracted_embeddings)  # Append embeddings for this frame

                # Display the frame with bounding boxes around detected faces
                for face in detector.detect_faces(frame):
                    x, y, w, h = face['box']
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                cv2.imshow('Video', frame)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                frame_counter += 1

            else:
                continue

        video_capture.release()
        cv2.destroyAllWindows()

        if single_video_embeddings != []:            
            one_face_videos[file_name] = single_video_embeddings
        else:
            one_face_videos[file_name] = 'too many faces'

                




1/1 [==============================] - 0s 46ms/step


In [17]:
single_video_embeddings

[[array([1.68396844e-04, 3.23298649e-04, 5.89292686e-05, 1.03829574e-04,
         5.13809900e-05, 5.12639876e-04, 9.12940686e-06, 3.49348738e-05,
         1.69002160e-05, 1.40978722e-04, 7.12474284e-04, 1.38774514e-04,
         5.77276733e-05, 7.96710956e-05, 1.63213554e-05, 4.84680495e-05,
         1.23409016e-04, 2.85986862e-05, 5.69476542e-05, 4.90671155e-05,
         1.93476561e-04, 1.61507109e-03, 9.65521554e-04, 2.19862355e-04,
         8.65635593e-05, 1.11264249e-04, 3.16551217e-04, 2.33811137e-04,
         1.72662229e-04, 1.89285711e-04, 4.23506499e-05, 3.27368907e-04,
         8.79582731e-05, 1.12511938e-04, 2.82565394e-04, 1.60456984e-05,
         1.22140569e-04, 1.08416052e-05, 5.88336866e-03, 3.70860507e-05,
         4.77840185e-05, 1.98933238e-04, 2.34981795e-04, 1.62409182e-04,
         5.45083312e-05, 1.24728584e-04, 3.96583819e-05, 2.28693621e-04,
         3.60279155e-05, 3.58025027e-05, 1.67388032e-04, 4.15605173e-04,
         2.92297918e-04, 5.17478620e-04, 2.49284029

In [4]:
df['faces'] = one_face_videos.values()

df = df[df.faces != 'too many faces']

# df['faces'] = df.faces.apply(lambda x: x)


In [62]:
df.faces.apply(lambda x: x[0])

array([[1.68950311e-04, 3.21029278e-04, 5.88095318e-05, 1.01736558e-04,
        5.08466474e-05, 5.20271540e-04, 9.06325840e-06, 3.48144640e-05,
        1.67241815e-05, 1.41240496e-04, 7.17859075e-04, 1.38363888e-04,
        5.75418198e-05, 7.94216467e-05, 1.62202414e-05, 4.82918331e-05,
        1.23148871e-04, 2.82507244e-05, 5.69789299e-05, 4.93185580e-05,
        1.92316074e-04, 1.62587897e-03, 9.72026319e-04, 2.20835966e-04,
        8.60042346e-05, 1.10360030e-04, 3.18247883e-04, 2.34424719e-04,
        1.71611784e-04, 1.90569306e-04, 4.20046345e-05, 3.22954787e-04,
        8.74409525e-05, 1.12908914e-04, 2.83601985e-04, 1.59982192e-05,
        1.22615413e-04, 1.08130198e-05, 5.88862086e-03, 3.69018744e-05,
        4.75762026e-05, 1.99742644e-04, 2.36423264e-04, 1.64945464e-04,
        5.42467969e-05, 1.26820931e-04, 3.94550334e-05, 2.30240708e-04,
        3.61259554e-05, 3.63780237e-05, 1.66541911e-04, 4.20553988e-04,
        2.90533935e-04, 5.12902101e-04, 2.51624442e-04, 2.387285

In [5]:
padding_length = len(df) + len(df.Emotion.unique()) + len(df.gender.unique())

def preprocess(text):
    t = text.lower()
    t = re.sub('\d+',r'',t)
    t = re.sub(r'\W+',r' ',t)
    return t

lemmatizer = WordNetLemmatizer()



df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(txt))])
                 .strip() for txt in df['Utterance']]


texts = df.prepro.values
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=padding_length, padding='post')

C:\Users\gsevr\AppData\Local\Temp\ipykernel_15788\790835697.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(txt))])


In [6]:
glove_path = 'glove.6B.50d.txt'
word_vectors = KeyedVectors.load_word2vec_format(glove_path, binary=False, encoding='utf8',no_header=True)



In [7]:
embedding_dim = 50  
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

embedding_layer = Embedding(
    input_dim=len(tokenizer.word_index) + 1,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=padding_length,  #for matmul
    trainable=False
)


model = Sequential()
model.add(embedding_layer)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics='accuracy')


In [8]:
embedding_vectors = model.predict(padded_sequences)
df['text'] = list(embedding_vectors)

1/1 [==============================] - 0s 36ms/step


C:\Users\gsevr\AppData\Local\Temp\ipykernel_15788\1340693711.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = list(embedding_vectors)


In [9]:
G = nx.Graph()


In [10]:

emotion_col = {
            'sadness':'teal',
            'surprise':'silver',
            'neutral':'lightgreen',
            'joy':'purple',
            'anger':'black',
            'disgust': 'brown',
            'fear':'yellow'
}

gender_col = {
    'male':'blue',
    'female':'pink'
}

In [11]:
for emotion in df.Emotion.unique():
    G.add_node(emotion, color=emotion_col[emotion],hover=emotion,emotion=emotion, type='emotion')


In [12]:
for gender in df.gender.unique():
    G.add_node(gender,color=gender_col[gender], hover=gender, gender=gender, type='gender')

In [13]:
for idx, row in df.iterrows():
    G.add_node(row['Utterance'], 
    text_embedding=row['text'],
    visual_embedding=row['faces'],
    color=emotion_col[row['Emotion']],
    hover=row['Utterance'],
    emotion=row['Emotion'],
    gender=row['gender'], 
    type='utterance',
    padding='zero')

In [14]:
for idx, row in df.iterrows():
    G.add_edge(row['Utterance'], row['Emotion'],color=emotion_col[row['Emotion']],weight=1)
    G.add_edge(row['Utterance'], row['gender'], color=gender_col[row['gender']],weight=1)


In [15]:
pos = nx.spring_layout(G)
# pos = nx.fruchterman_reingold_layout(G)


In [16]:
import textwrap

fig = go.Figure()

node_positions = {}

for node, position in pos.items():
    x, y = position  

    color = G.nodes[node]['color']  
    hover = G.nodes[node]['hover']
    hover = '<br>'.join(textwrap.wrap(hover,width=50))
    try:
        emotion = G.nodes[node]['emotion']
    except KeyError:
        try:
            gender = G.nodes[node]['gender']
        except KeyError:
            continue
    node_type = G.nodes[node]['type']

    if node_type == 'utterance':
        fig.add_trace(go.Scatter(x=[x], y=[y], marker=dict(size=10,color=color), hovertext=hover, name=emotion, hovertemplate=hover))
    else:
        fig.add_trace(go.Scatter(x=[x], y=[y], marker=dict(size=10,color=color), hovertext=hover, name=node_type, hovertemplate=hover))


for edge in G.edges():
    color = G.edges[edge]['color']
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    fig.add_trace(go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines', line=dict(width=1,color=color), name='Edge'))

fig.update_layout(showlegend=False, title='Emotions Graph')

fig.update_layout(xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

fig.show()

In [17]:
# Adj matrix

adjacency_info = dict(G.adj)

nodes = list(G.nodes())

adj_matrix = np.zeros((len(nodes), len(nodes)))

for i, node in enumerate(nodes):
    for j, other_node in enumerate(nodes):
        if other_node in adjacency_info[node]:
            adj_matrix[i, j] = 1 

print(adj_matrix)


[[0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [18]:
# try spektral

adjacency_matrix = nx.adjacency_matrix(G)  # Adjacency matrix
features_text = np.array([G.nodes[node]['text_embedding'] for node in G.nodes() if G.nodes[node]['type'] == 'utterance']).T  # 'text' features, perm
features_video = np.array([G.nodes[node]['visual_embedding'] for node in G.nodes() if G.nodes[node]['type'] == 'utterance'])  # 'video' features

adj_normalized = normalized_laplacian(adjacency_matrix)



C:\Users\gsevr\AppData\Local\Temp\ipykernel_15788\1966725035.py:5: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [72]:
np.array(features_video[0]).shape

(3, 1, 1, 1000)

In [19]:
y_train = [G.nodes[node]['emotion'] for node in G.nodes if G.nodes[node]['type'] == 'emotion']

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

y_train = ohe.fit_transform(np.array(y_train).reshape(-1,1))

y_train = tf.constant(y_train.toarray())

In [20]:
# features_text = tf.constant(np.pad(features_text,((0,5))))
# features_video = tf.constant(np.pad(features_video,((0,5))))
# y_train = tf.constant(np.pad(y_train,((0,9))))

In [73]:
text_in = Input(shape=(features_text.shape[1],),name='text_input_layer')
video_in = Input(shape=(np.array(features_video[0]).shape[1]),name='video_input_layer')
adj_in = Input((adj_matrix.shape[0],), sparse=True, name='adj_matrix')

In [74]:
from tensorflow.keras.layers import Permute

gcn1 = GCNConv(16, activation='relu')([text_in, adj_in])
gcn1 = Dropout(0.5)(gcn1)
gcn2 = GCNConv(8, activation='relu')([gcn1, adj_in])


In [75]:
merged_features = Concatenate()([gcn2, video_in])

In [76]:
fc = Dense(16, activation='relu')(merged_features)
output = Dense(len(df.Emotion.unique()), activation='softmax')(fc)

In [77]:
model = Model(inputs=[text_in, video_in, adj_in], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [78]:
print(features_text.shape)
print(features_video.shape)
print(y_train.shape)
print(adj_normalized.shape)

(50, 16, 10)
(10,)
(4, 4)
(16, 16)


In [79]:
transpose_text = tf.transpose(features_text,perm=[2,0,1])
print(transpose_text.shape)
transpose_video = tf.reshape(features_video,(50,20,features_video.shape[0]))
transpose_video = tf.transpose(transpose_video,perm=[2,0,1])
print(transpose_video.shape)



(10, 50, 16)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [80]:
history = model.fit([features_text, features_video, tf.constant(adj_normalized.toarray())],
          y_train,
          epochs=100,
          batch_size=32,
          )

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).